# Задание 2

### Инициализация окружения
В качестве документации для задания два выступает данный документ Jupyter Notebook. В нём записаны все скрипты и SQL-запросы, касающиеся пунктов 2.1 и 2.2.
venv для данного задания и задания 3 одинаковый, поэтому на данном этапе можно задействовать некоторые из необходимых далее библиотек.

In [ ]:
%pip install sqlalchemy psycopg2-binary pandas

Далее следует создать соединение к базе данных, через которое будут протестированы все последующие запросы.
Необходимые данные для соединения с БД должны браться хотя бы из .env-файла, но в рамках Jupyter Notebook все параметры внесены в отдельный словарь.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Параметры подключения к БД
DB_CONFIG = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'main_db'
}

# Создаём движок SQLAlchemy
engine = create_engine(
            f"{DB_CONFIG['drivername']}://{DB_CONFIG['username']}:{DB_CONFIG['password']}@"
            f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        )
# Создаём соединение с БД на основе движка
db_connection = engine.connect()
db_connection.closed # Если выведен False, то соединение настроено

False

### Задание 2.1.
Для данного задания необходимо получить информацию о сумме товаров, заказанных под каждого клиента.
Получение суммы реализовано через последовательность запросов JOIN вместе с функцией COALESCE, позволяющей вывести сумму 0 в случае отсутствия заказов у данного клиента.
Вместо обычного JOIN используется LEFT JOIN для включения в вывод клиентов, которые ещё не сделали ни одного заказа.

In [14]:
task_2_1_query = """
SELECT 
    c.name AS client_name,
    COALESCE(SUM(og.amount), 0) AS total_amount
FROM Clients c
LEFT JOIN Orders o ON o.client_id = c.id
LEFT JOIN Ordered_goods og ON og.order_id = o.id
LEFT JOIN Goods g ON g.id = og.good_id
GROUP BY c.id, c.name
ORDER BY total_amount DESC;
"""

# Выполняем запрос на получения данных из БД
df = pd.read_sql(task_2_1_query, engine)
df

,client_name,total_amount
0,Козлова Ирина Петровна,29
1,Кузнецов Алексей Викторович,29
2,Захаров Сергей Викторович,29
3,Петров Петр Петрович,23
4,Смирнова Ольга Владимировна,20
5,Иванов Иван Иванович,17
6,Волков Андрей Сергеевич,13
7,Сидорова Мария Сергеевна,8
8,Николаева Екатерина Александровна,2
9,Федоров Дмитрий Николаевич,0


При этом может потребоваться информация о суммарной стоимости всех заказанных товарах, которое получается простой заменой выражения внутри функции SUM.

In [15]:
# Заменяем выражение внутри функции SUM
sum_price_query = task_2_1_query.replace("SUM(og.amount)", "SUM(g.price * og.amount)")

# Заменяем все упоминания total_amount на total_cost
sum_price_query = sum_price_query.replace("total_amount", "total_cost")

df = pd.read_sql(sum_price_query, engine)
df

,client_name,total_cost
0,Козлова Ирина Петровна,1817971.9
1,Кузнецов Алексей Викторович,1794971.9
2,Захаров Сергей Викторович,1731471.9
3,Петров Петр Петрович,1335977.9
4,Смирнова Ольга Владимировна,1187480.9
5,Иванов Иван Иванович,830484.8
6,Волков Андрей Сергеевич,804987.9
7,Сидорова Мария Сергеевна,449992.0
8,Николаева Екатерина Александровна,128998.0
9,Федоров Дмитрий Николаевич,0.0
